In [1]:
! hdfs dfs -ls 

Found 1 items
drwxr-xr-x   - jovyan supergroup          0 2018-10-21 17:41 wc_mr


In [2]:
! hdfs dfs -ls -R /data

drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part
-rwxrwxrwx   1 jovyan supergroup   76861985 2017-10-17 13:15 /data/wiki/en_articles_part/articles-part


In [16]:
from __future__ import print_function

### Mapper: simple

In [39]:
%%writefile mapper_wc.py
#!/usr/bin/env python

from __future__ import print_function
import sys

for line in sys.stdin:
    article_id, content = line.split('\t', 1)
    words = content.split()
    for word in words:
        print(word, 1, sep='\t')
        

Overwriting mapper_wc.py


In [29]:
!cat mapper_wc.py

#!/usr/bin/env python

from __future__ import print_function
import sys

for line in sys.stdin:
    article_id, content = line.split('\t', 1)
    words = content.split()
    for word in words:
        print(word, 1, sep='\t')
        

In [30]:
%%bash

OUT_DIR=wc_mr

hdfs dfs -rm -r -skipTrash ${OUT_DIR}

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper_wc.py\
    -mapper mapper_wc.py \
    -numReduceTasks 0 \
    -input /data/wiki/en_articles_part \
    -output $OUT_DIR


Deleted wc_mr
packageJobJar: [/tmp/hadoop-unjar6320021201903899629/] [] /tmp/streamjob5999991034133979879.jar tmpDir=null


18/10/21 18:03:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/21 18:03:37 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/21 18:03:38 INFO mapred.FileInputFormat: Total input files to process : 1
18/10/21 18:03:38 INFO mapreduce.JobSubmitter: number of splits:2
18/10/21 18:03:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1540138670744_0014
18/10/21 18:03:39 INFO impl.YarnClientImpl: Submitted application application_1540138670744_0014
18/10/21 18:03:39 INFO mapreduce.Job: The url to track the job: http://21ad51e5bf54:8088/proxy/application_1540138670744_0014/
18/10/21 18:03:39 INFO mapreduce.Job: Running job: job_1540138670744_0014
18/10/21 18:03:51 INFO mapreduce.Job: Job job_1540138670744_0014 running in uber mode : false
18/10/21 18:03:51 INFO mapreduce.Job:  map 0% reduce 0%
18/10/21 18:04:12 INFO mapreduce.Job:  map 74% reduce 0%
18/10/21 18:04:14 INFO mapreduce.Job:  map 84% reduce 0%
18/10/21 18:04:16 INFO 

In [32]:
! hdfs dfs -cat wc_mr/* | head 

Basel	1
Basel	1
(	1
)	1
or	1
Basle	1
(	1
;	1
;	1
)	1
cat: Unable to write to output stream.
cat: Unable to write to output stream.


#### check mapper

In [33]:
%%bash

OUT_DIR=wc_mr

hdfs dfs -rm -r -skipTrash ${OUT_DIR}

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper_wc.py\
    -mapper mapper_wc.py \
    -numReduceTasks 1 \
    -input /data/wiki/en_articles_part \
    -output $OUT_DIR


Deleted wc_mr
packageJobJar: [/tmp/hadoop-unjar5672727913801196451/] [] /tmp/streamjob7255745284088310442.jar tmpDir=null


18/10/21 18:08:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/21 18:08:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/21 18:08:33 INFO mapred.FileInputFormat: Total input files to process : 1
18/10/21 18:08:33 INFO mapreduce.JobSubmitter: number of splits:2
18/10/21 18:08:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1540138670744_0015
18/10/21 18:08:34 INFO impl.YarnClientImpl: Submitted application application_1540138670744_0015
18/10/21 18:08:34 INFO mapreduce.Job: The url to track the job: http://21ad51e5bf54:8088/proxy/application_1540138670744_0015/
18/10/21 18:08:34 INFO mapreduce.Job: Running job: job_1540138670744_0015
18/10/21 18:08:45 INFO mapreduce.Job: Job job_1540138670744_0015 running in uber mode : false
18/10/21 18:08:45 INFO mapreduce.Job:  map 0% reduce 0%
18/10/21 18:09:06 INFO mapreduce.Job:  map 34% reduce 0%
18/10/21 18:09:12 INFO mapreduce.Job:  map 46% reduce 0%
18/10/21 18:09:24 INFO 

In [34]:
! hdfs dfs -cat wc_mr/* | head 

!	1
!	1
!	1
!	1
!	1
!	1
!	1
!	1
!	1
!	1
cat: Unable to write to output stream.


### Mapper: right

In [36]:
import re

In [40]:
%%writefile mapper_wc.py
#!/usr/bin/env python

from __future__ import print_function
import sys
import re

for line in sys.stdin:
    article_id, content = line.split('\t', 1)
    words = re.split("\W+", content)
    for word in words:
        print(word, 1, sep='\t')
        

Overwriting mapper_wc.py


In [41]:
!cat mapper_wc.py

#!/usr/bin/env python

from __future__ import print_function
import sys
import re

for line in sys.stdin:
    article_id, content = line.split('\t', 1)
    words = re.split("\W+", content)
    for word in words:
        print(word, 1, sep='\t')
        

In [42]:
%%bash

OUT_DIR=wc_mr

hdfs dfs -rm -r -skipTrash ${OUT_DIR}

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -files mapper_wc.py\
    -mapper mapper_wc.py \
    -numReduceTasks 1 \
    -input /data/wiki/en_articles_part \
    -output $OUT_DIR


Deleted wc_mr
packageJobJar: [/tmp/hadoop-unjar3216437397648960962/] [] /tmp/streamjob9138721520562476023.jar tmpDir=null


18/10/21 18:40:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/21 18:40:40 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/10/21 18:41:42 INFO mapred.FileInputFormat: Total input files to process : 1
18/10/21 18:41:58 INFO mapreduce.JobSubmitter: number of splits:2
18/10/21 18:42:27 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1540138670744_0016
18/10/21 18:50:42 INFO impl.YarnClientImpl: Submitted application application_1540138670744_0016
18/10/21 18:50:42 INFO mapreduce.Job: The url to track the job: http://21ad51e5bf54:8088/proxy/application_1540138670744_0016/
18/10/21 18:50:42 INFO mapreduce.Job: Running job: job_1540138670744_0016
18/10/21 18:51:06 INFO mapreduce.Job: Job job_1540138670744_0016 running in uber mode : false
18/10/21 18:51:06 INFO mapreduce.Job:  map 0% reduce 0%
18/10/21 18:51:06 INFO mapreduce.Job: Job job_1540138670744_0016 failed with state FAILED due to: Application application_1540138670744_0

In [43]:
! hdfs dfs -cat wc_mr/* | head 

cat: `wc_mr/*': No such file or directory
